In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

import matplotlib.pyplot as plot

import numpy as np

# Reading the Dataset

In [6]:
df = pd.read_csv("rawdata.csv")
print(df.shape)

(1146, 20)


# Fix Map most used Map
###### We want to analyze the different combinations of brawler types. As the combinations depend on the map, we fix our data frame to th most used map.
- first check which map is used the most

In [7]:
pd.value_counts(df['map'])

Backyard Bowl     903
Post Haste        119
Warped Arena       25
Triple Dribble     22
Super Stadium      17
Beach Ball         16
Field Goal         11
Pinhole Punt        9
Well Cut            7
Pinball Dreams      6
Penalty Kick        4
Galaxy Arena        4
Center Field        2
Shoulder Bash       1
Name: map, dtype: int64

- copy initial data frame and remove all games that are played on other maps

In [8]:
df_fixed_map = df.copy()
df_fixed_map = df_fixed_map[df_fixed_map['map'] == 'Backyard Bowl'] 

# Define Brawler Type Lists
###### We need to define the type for each brawler. The information is obtained from the game itself.

In [9]:

sniper_brawlers = ['COLT','RICO','8-BIT','PIPER','BROCK','BEA','PENNY','MR. P','SPIKE']
thrower_brawlers = ['DYNAMIKE','TICK','BARLEY']
heavyweight_brawlers = ['DARRYL','EL PRIMO','ROSA','BULL','FRANK','JACKY','BIBI']
fighter_brawlers = ['CARL','EMZ','BO','TARA','JESSIE','NITA','SHELLY']
support_brawlers = ['POCO','PAM','GENE','MAX','SPROUT','SANDY','GALE',]
assassin_brawlers = ['MORTIS','CROW','LEON']


# Add Brawler Types to the DataFrame
### Function to Assign Brawler Type

In [10]:
def assign_brawler_type(brawler):
    if brawler in sniper_brawlers:
        return 'SNIPER'
    elif brawler in thrower_brawlers:
        return 'THROWER'
    elif brawler in heavyweight_brawlers:
        return 'HEAVYWEIGHT'
    elif brawler in fighter_brawlers:
        return 'FIGHTER'
    elif brawler in support_brawlers:
        return 'SUPPORTER'
    elif brawler in assassin_brawlers:
        return 'ASSASSIN'
    else:
        return ''
    

### Add Brawler Types to DataFrame

In [11]:
df_fixed_map['Brawler1Type'] = df_fixed_map['Brawler1Name'].apply(assign_brawler_type)
df_fixed_map['Brawler2Type'] = df_fixed_map['Brawler2Name'].apply(assign_brawler_type)
df_fixed_map['Brawler3Type'] = df_fixed_map['Brawler3Name'].apply(assign_brawler_type)
df_fixed_map['Brawler4Type'] = df_fixed_map['Brawler4Name'].apply(assign_brawler_type)
df_fixed_map['Brawler5Type'] = df_fixed_map['Brawler5Name'].apply(assign_brawler_type)
df_fixed_map['Brawler6Type'] = df_fixed_map['Brawler6Name'].apply(assign_brawler_type)


# Check if same Type is used by multiple Players
###### If used by multiple players we add _<<number>> to the type to be able to compute the frequent itemsets

In [12]:
counter = 0
for index, row in df_fixed_map.iterrows():
    if row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] != row['Brawler3Type']:
        df_fixed_map.loc[index, 'Brawler2Type'] = row['Brawler2Type']+'_2'
    elif row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] == row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler2Type'] = row['Brawler2Type']+'_2'
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_3'
    elif row['Brawler1Type'] == row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] != row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
    elif row['Brawler1Type'] != row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] == row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
df_fixed_map.head(20)



WinningTeam            BattleTime  playerTag       mode            map  \
0             1  20200530T122433.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
1             1  20200530T121808.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
2             1  20200530T121207.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
3             1  20200530T120359.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
4             1  20200530T120028.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
5             1  20200530T115600.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
6             2  20200530T115304.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
7             2  20200530T115056.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
8             2  20200530T114747.000Z   #9U2QJ0P  brawlBall  Backyard Bowl   
11            2  20200530T143201.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
12            1  20200530T142911.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
13            2  20200530T142430.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
14            1  20200530T142017.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
15            2  20200530T141625.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
16            2  20200530T140826.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
17            2  20200530T140427.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
18            2  20200530T140031.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
19            1  20200530T135358.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
20            1  20200530T135012.000Z  #8P80J8RC  brawlBall  Backyard Bowl   
21            2  20200530T134504.000Z  #8P80J8RC  brawlBall  Backyard Bowl   

    result StarPlayerTag StarPlayerBrawler Brawler1Name Brawler1Tag  ...  \
0        1      #9U2QJ0P               BEA        PIPER   #VGR0CJ29  ...   
1        1      #9U2QJ0P               BEA        PIPER   #VGR0CJ29  ...   
2        1      #9U2QJ0P               BEA        PIPER   #VGR0CJ29  ...   
3        1     #VGR0CJ29             PIPER        PIPER   #VGR0CJ29  ...   
4        1     #VGR0CJ29             PIPER        PIPER   #VGR0CJ29  ...   
5        1    #9UR09Q80Y              GENE         GENE  #9UR09Q80Y  ...   
6        1     #VGR0CJ29             PIPER          PAM  #20JRL29PY  ...   
7        1     #VGR0CJ29             PIPER        PIPER  #20JLGV0QL  ...   
8        1    #9UR09Q80Y              GENE          BEA   #PJJ9P890  ...   
11      -1     #YCC8VGRU               PAM        MR. P   #8P80J8RC  ...   
12       1     #UQY8PCGP               BEA        BROCK     #PQRVCC  ...   
13      -1     #8LC2CJY2             MR. P        MR. P   #8P80J8RC  ...   
14       1     #8P80J8RC             MR. P        BROCK     #PQRVCC  ...   
15       1     #8P80J8RC             MR. P          BEA   #UUUL2PLY  ...   
16       1       #PQRVCC             BROCK        SANDY  #8V8QYV0P0  ...   
17       1     #8P80J8RC             MR. P        JACKY  #8LUY9PPRL  ...   
18       1     #8P80J8RC             MR. P          PAM    #R28VYC9  ...   
19       1       #PQRVCC             BROCK        BROCK     #PQRVCC  ...   
20       1       #PQRVCC             BROCK        BROCK     #PQRVCC  ...   
21       1       #PQRVCC             BROCK          BEA   #2VL2CCCG  ...   

   Brawler5Name Brawler5Tag Brawler6Name Brawler6Tag Brawler1Type  \
0         BROCK  #20C2RRRGY          EMZ   #Y220JJJ9       SNIPER   
1           MAX  #22L8CLYQU        PENNY  #28U009JPY       SNIPER   
2         BROCK    #9QJP9LP        SPIKE  #98VJCL2PQ       SNIPER   
3         FRANK  #2UCV2R0LV         POCO  #22RURYRV0       SNIPER   
4         BROCK  #98JQVGLP9        JACKY  #8822ULJ9Q       SNIPER   
5         PIPER  #298R28UG2        BROCK   #QRVCCPPR    SUPPORTER   
6          GENE  #9UR09Q80Y          BEA    #9U2QJ0P    SUPPORTER   
7           BEA    #9U2QJ0P         GENE  #9UR09Q80Y       SNIPER   
8           BEA    #9U2QJ0P         GENE  #9UR09Q80Y       SNIPER   
11          PAM   #YCC8VGRU        PIPER   #9JY82J8U       SNIPER   
12       

# Find Frequent Combinations of Brawler Types
###### First we want to find the frequent used combinations of brawler types. This is done by computing the frequent itemsets.

In [13]:
combinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team1 = []
    team2 = []
    team1.append(row["Brawler1Type"])
    team1.append(row["Brawler2Type"])
    team1.append(row["Brawler3Type"])
    team2.append(row["Brawler4Type"])
    team2.append(row["Brawler5Type"])
    team2.append(row["Brawler6Type"])

    combinations.append(team1)
    combinations.append(team2)

In [14]:
te = TransactionEncoder()
te_ary = te.fit(combinations).transform(combinations)
df_fixed_mapFreqSetsCombinations = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsCombinations


ASSASSIN  ASSASSIN_2  ASSASSIN_3  FIGHTER  FIGHTER_2  FIGHTER_3  \
0        False       False       False    False      False      False   
1        False       False       False     True      False      False   
2        False       False       False    False      False      False   
3        False       False       False    False      False      False   
4        False       False       False    False      False      False   
...        ...         ...         ...      ...        ...        ...   
1801     False       False       False     True      False      False   
1802     False       False       False     True       True      False   
1803      True       False       False    False      False      False   
1804     False       False       False     True       True      False   
1805      True       False       False    False      False      False   

      HEAVYWEIGHT  HEAVYWEIGHT_2  HEAVYWEIGHT_3  SNIPER  SNIPER_2  SNIPER_3  \
0           False          False          False    True      True     False   
1           False          False          False    True     False     False   
2           False          False          False    True      True     False   
3           False          False          False    True     False     False   
4           False          False          False    True      True     False   
...           ...            ...            ...     ...       ...       ...   
1801        False          False          False    True     False     False   
1802        False          False          False    True     False     False   
1803         True          False          False   False     False     False   
1804        False          False          False    True     False     False   
1805        False          False          False    True     False     False   

      SUPPORTER  SUPPORTER_2  SUPPORTER_3  THROWER  
0          True        False        False    False  
1         False        False        False    False  
2          True        False        False    False  
3          True        False        False    False  
4          True        False        False    False  
...         ...          ...          ...      ...  
1801       True        False        False    False  
1802      False        False        False    False  
1803       True        False        False    False  
1804      False        False        False    False  
1805       True        False        False    False  

[1806 rows x 16 columns]

In [15]:
frequent_itemsets_combinations = apriori(df_fixed_mapFreqSetsCombinations, min_support=0.001, use_colnames=True)
frequent_itemsets_combinations['length'] = frequent_itemsets_combinations['itemsets'].apply(lambda x: len(x))

In [16]:
frequent_itemsets_combinations[ (frequent_itemsets_combinations['length'] == 3) ].sort_values(by=["support"], ascending=False)




support                                 itemsets  length
87   0.075305             (SUPPORTER, FIGHTER, SNIPER)       3
75   0.073090            (SUPPORTER, SNIPER, ASSASSIN)       3
97   0.059801            (SUPPORTER, SNIPER_2, SNIPER)       3
96   0.052049             (SNIPER_2, SNIPER, SNIPER_3)       3
93   0.047619         (SUPPORTER, HEAVYWEIGHT, SNIPER)       3
74   0.035991             (SNIPER_2, SNIPER, ASSASSIN)       3
67   0.035437              (FIGHTER, SNIPER, ASSASSIN)       3
98   0.033776         (SUPPORTER, SUPPORTER_2, SNIPER)       3
71   0.027132          (HEAVYWEIGHT, SNIPER, ASSASSIN)       3
72   0.024917       (SUPPORTER, HEAVYWEIGHT, ASSASSIN)       3
68   0.024917           (SUPPORTER, FIGHTER, ASSASSIN)       3
66   0.020487         (FIGHTER, HEAVYWEIGHT, ASSASSIN)       3
83   0.019380           (FIGHTER, HEAVYWEIGHT, SNIPER)       3
84   0.018272        (SUPPORTER, FIGHTER, HEAVYWEIGHT)       3
86   0.017165              (FIGHTER, SNIPER_2, SNIPER)       3
63   0.011628           (ASSASSIN_2, SNIPER, ASSASSIN)       3
100  0.011628    (SUPPORTER, SUPPORTER_2, SUPPORTER_3)       3
77   0.009413       (SUPPORTER, SUPPORTER_2, ASSASSIN)       3
89   0.009413        (SUPPORTER, FIGHTER, SUPPORTER_2)       3
92   0.008859          (HEAVYWEIGHT, SNIPER_2, SNIPER)       3
80   0.008306             (FIGHTER, FIGHTER_2, SNIPER)       3
65   0.005537           (FIGHTER, FIGHTER_2, ASSASSIN)       3
70   0.004983   (HEAVYWEIGHT_2, HEAVYWEIGHT, ASSASSIN)       3
95   0.004983    (SUPPORTER, SUPPORTER_2, HEAVYWEIGHT)       3
99   0.004983             (SUPPORTER, THROWER, SNIPER)       3
64   0.004430        (SUPPORTER, ASSASSIN_2, ASSASSIN)       3
61   0.003322          (FIGHTER, ASSASSIN_2, ASSASSIN)       3
62   0.002769      (ASSASSIN_2, HEAVYWEIGHT, ASSASSIN)       3
90   0.002769     (HEAVYWEIGHT_2, HEAVYWEIGHT, SNIPER)       3
91   0.002215  (SUPPORTER, HEAVYWEIGHT, HEAVYWEIGHT_2)       3
81   0.002215          (SUPPORTER, FIGHTER, FIGHTER_2)       3
85   0.002215          (FIGHTER, THROWER, HEAVYWEIGHT)       3
78   0.002215          (FIGHTER, FIGHTER_2, FIGHTER_3)       3
88   0.001661               (FIGHTER, THROWER, SNIPER)       3
94   0.001661           (HEAVYWEIGHT, THROWER, SNIPER)       3
79   0.001661        (FIGHTER, FIGHTER_2, HEAVYWEIGHT)       3
76   0.001661              (THROWER, SNIPER, ASSASSIN)       3
101  0.001661        (SUPPORTER, SUPPORTER_2, THROWER)       3
82   0.001107    (HEAVYWEIGHT_2, FIGHTER, HEAVYWEIGHT)       3
73   0.001107         (THROWER, HEAVYWEIGHT, ASSASSIN)       3
69   0.001107             (FIGHTER, THROWER, ASSASSIN)       3

# Find Frequent Winning Combinations of Brawler Types
###### We want to find the frequent winning combinations of brawler types. This is done by computing the frequent itemsets.

In [17]:
winningCombinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])
    elif row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])

    if len(team) != 0:
        winningCombinations.append(team)

In [18]:
te = TransactionEncoder()
te_ary = te.fit(winningCombinations).transform(winningCombinations)
df_fixed_mapFreqSetsWinner = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsWinner

ASSASSIN  ASSASSIN_2  FIGHTER  FIGHTER_2  FIGHTER_3  HEAVYWEIGHT  \
0       False       False    False      False      False        False   
1       False       False    False      False      False        False   
2       False       False    False      False      False        False   
3       False       False    False      False      False        False   
4       False       False    False      False      False        False   
..        ...         ...      ...        ...        ...          ...   
868     False       False     True      False      False        False   
869     False       False     True      False      False        False   
870     False       False     True       True      False        False   
871     False       False     True       True      False        False   
872     False       False     True       True      False        False   

     HEAVYWEIGHT_2  SNIPER  SNIPER_2  SNIPER_3  SUPPORTER  SUPPORTER_2  \
0            False    True      True     False       True        False   
1            False    True      True     False       True        False   
2            False    True      True     False       True        False   
3            False    True      True     False       True        False   
4            False    True      True     False       True        False   
..             ...     ...       ...       ...        ...          ...   
868          False    True     False     False      False        False   
869          False    True     False     False      False        False   
870          False    True     False     False      False        False   
871          False    True     False     False      False        False   
872          False    True     False     False      False        False   

     SUPPORTER_3  THROWER  
0          False    False  
1          False    False  
2          False    False  
3          False    False  
4          False    False  
..           ...      ...  
868        False    False  
869        False    False  
870        False    False  
871        False    False  
872        False    False  

[873 rows x 14 columns]

In [19]:
frequent_itemsets_winner = apriori(df_fixed_mapFreqSetsWinner, min_support=0.001, use_colnames=True)
frequent_itemsets_winner['length'] = frequent_itemsets_winner['itemsets'].apply(lambda x: len(x))

In [20]:
frequent_itemsets_winner[ (frequent_itemsets_winner['length'] == 3) ].sort_values(by=["support"], ascending=False)

support                                 itemsets  length
88  0.077892             (SNIPER_2, SNIPER, SNIPER_3)       3
80  0.067583             (SUPPORTER, FIGHTER, SNIPER)       3
68  0.058419            (SUPPORTER, SNIPER, ASSASSIN)       3
89  0.058419            (SUPPORTER, SNIPER_2, SNIPER)       3
86  0.046964         (SUPPORTER, HEAVYWEIGHT, SNIPER)       3
67  0.036655             (SNIPER_2, SNIPER, ASSASSIN)       3
90  0.035510         (SUPPORTER, SUPPORTER_2, SNIPER)       3
63  0.029782              (FIGHTER, SNIPER, ASSASSIN)       3
66  0.017182       (SUPPORTER, HEAVYWEIGHT, ASSASSIN)       3
92  0.016037    (SUPPORTER, SUPPORTER_2, SUPPORTER_3)       3
65  0.013746          (SNIPER, HEAVYWEIGHT, ASSASSIN)       3
79  0.011455              (FIGHTER, SNIPER_2, SNIPER)       3
85  0.011455          (SNIPER_2, HEAVYWEIGHT, SNIPER)       3
64  0.011455           (SUPPORTER, FIGHTER, ASSASSIN)       3
74  0.010309             (FIGHTER, FIGHTER_2, SNIPER)       3
77  0.008018        (SUPPORTER, FIGHTER, HEAVYWEIGHT)       3
76  0.008018           (FIGHTER, HEAVYWEIGHT, SNIPER)       3
70  0.008018       (SUPPORTER, SUPPORTER_2, ASSASSIN)       3
62  0.006873         (FIGHTER, HEAVYWEIGHT, ASSASSIN)       3
82  0.005727        (SUPPORTER, FIGHTER, SUPPORTER_2)       3
91  0.004582             (SUPPORTER, THROWER, SNIPER)       3
60  0.003436           (ASSASSIN_2, SNIPER, ASSASSIN)       3
87  0.002291    (SUPPORTER, SUPPORTER_2, HEAVYWEIGHT)       3
58  0.002291          (FIGHTER, ASSASSIN_2, ASSASSIN)       3
78  0.002291          (FIGHTER, THROWER, HEAVYWEIGHT)       3
69  0.002291              (THROWER, SNIPER, ASSASSIN)       3
61  0.002291        (SUPPORTER, ASSASSIN_2, ASSASSIN)       3
93  0.002291        (SUPPORTER, SUPPORTER_2, THROWER)       3
81  0.001145               (FIGHTER, THROWER, SNIPER)       3
83  0.001145     (HEAVYWEIGHT_2, HEAVYWEIGHT, SNIPER)       3
84  0.001145  (SUPPORTER, HEAVYWEIGHT, HEAVYWEIGHT_2)       3
59  0.001145      (ASSASSIN_2, HEAVYWEIGHT, ASSASSIN)       3
75  0.001145          (SUPPORTER, FIGHTER, FIGHTER_2)       3
73  0.001145        (FIGHTER, FIGHTER_2, HEAVYWEIGHT)       3
72  0.001145          (FIGHTER, FIGHTER_2, FIGHTER_3)       3
71  0.001145           (SUPPORTER, THROWER, ASSASSIN)       3

# Find Frequent Loosing Combinations of Brawler Types
###### We want to find the frequent loosing combinations of brawler types. This is done by computing the frequent itemsets.

In [21]:
loosingCombinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])
    elif row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])

    if len(team) != 0:
        loosingCombinations.append(team)

In [22]:
te = TransactionEncoder()
te_ary = te.fit(loosingCombinations).transform(loosingCombinations)
df_fixed_mapFreqSetsLooser = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsLooser

ASSASSIN  ASSASSIN_2  ASSASSIN_3  FIGHTER  FIGHTER_2  FIGHTER_3  \
0       False       False       False     True      False      False   
1       False       False       False    False      False      False   
2       False       False       False    False      False      False   
3       False       False       False    False      False      False   
4       False       False       False    False      False      False   
..        ...         ...         ...      ...        ...        ...   
868      True       False       False    False      False      False   
869     False       False       False     True      False      False   
870     False       False       False     True      False      False   
871      True       False       False    False      False      False   
872      True       False       False    False      False      False   

     HEAVYWEIGHT  HEAVYWEIGHT_2  HEAVYWEIGHT_3  SNIPER  SNIPER_2  SNIPER_3  \
0          False          False          False    True     False     False   
1          False          False          False    True     False     False   
2          False          False          False    True     False     False   
3           True          False          False   False     False     False   
4           True          False          False    True     False     False   
..           ...            ...            ...     ...       ...       ...   
868        False          False          False    True      True     False   
869         True          False          False   False     False     False   
870        False          False          False    True     False     False   
871         True          False          False   False     False     False   
872        False          False          False    True     False     False   

     SUPPORTER  SUPPORTER_2  SUPPORTER_3  THROWER  
0        False        False        False    False  
1         True        False        False    False  
2         True        False        False    False  
3         True        False        False    False  
4        False        False        False    False  
..         ...          ...          ...      ...  
868      False        False        False    False  
869       True        False        False    False  
870       True        False        False    False  
871       True        False        False    False  
872       True        False        False    False  

[873 rows x 16 columns]

In [23]:
frequent_itemsets_looser = apriori(df_fixed_mapFreqSetsLooser, min_support=0.001, use_colnames=True)
frequent_itemsets_looser['length'] = frequent_itemsets_looser['itemsets'].apply(lambda x: len(x))

In [24]:
frequent_itemsets_looser[ (frequent_itemsets_looser['length'] == 3) ].sort_values(by=["support"], ascending=False)



support                                     itemsets  length
83   0.087056                (SUPPORTER, SNIPER, ASSASSIN)       3
95   0.080183                 (SUPPORTER, FIGHTER, SNIPER)       3
107  0.063001                (SUPPORTER, SNIPER_2, SNIPER)       3
102  0.049255             (SUPPORTER, HEAVYWEIGHT, SNIPER)       3
75   0.043528                  (FIGHTER, SNIPER, ASSASSIN)       3
79   0.042383              (HEAVYWEIGHT, SNIPER, ASSASSIN)       3
76   0.036655               (SUPPORTER, FIGHTER, ASSASSIN)       3
82   0.036655                 (SNIPER_2, SNIPER, ASSASSIN)       3
74   0.035510             (FIGHTER, HEAVYWEIGHT, ASSASSIN)       3
109  0.032073             (SUPPORTER, SUPPORTER_2, SNIPER)       3
91   0.030928               (FIGHTER, HEAVYWEIGHT, SNIPER)       3
80   0.030928           (SUPPORTER, HEAVYWEIGHT, ASSASSIN)       3
92   0.029782            (SUPPORTER, FIGHTER, HEAVYWEIGHT)       3
106  0.022910                 (SNIPER_2, SNIPER, SNIPER_3)       3
94   0.022910                  (FIGHTER, SNIPER_2, SNIPER)       3
71   0.020619               (ASSASSIN_2, SNIPER, ASSASSIN)       3
97   0.012600            (SUPPORTER, FIGHTER, SUPPORTER_2)       3
73   0.011455               (FIGHTER, FIGHTER_2, ASSASSIN)       3
78   0.010309       (HEAVYWEIGHT_2, HEAVYWEIGHT, ASSASSIN)       3
85   0.010309           (SUPPORTER, SUPPORTER_2, ASSASSIN)       3
104  0.006873        (SUPPORTER, SUPPORTER_2, HEAVYWEIGHT)       3
101  0.006873              (HEAVYWEIGHT, SNIPER_2, SNIPER)       3
88   0.006873                 (FIGHTER, FIGHTER_2, SNIPER)       3
72   0.006873            (SUPPORTER, ASSASSIN_2, ASSASSIN)       3
111  0.006873        (SUPPORTER, SUPPORTER_2, SUPPORTER_3)       3
110  0.005727                 (SUPPORTER, THROWER, SNIPER)       3
70   0.004582          (ASSASSIN_2, HEAVYWEIGHT, ASSASSIN)       3
69   0.003436              (FIGHTER, ASSASSIN_2, ASSASSIN)       3
89   0.003436              (SUPPORTER, FIGHTER, FIGHTER_2)       3
103  0.003436               (HEAVYWEIGHT, THROWER, SNIPER)       3
100  0.003436      (SUPPORTER, HEAVYWEIGHT, HEAVYWEIGHT_2)       3
99   0.003436         (HEAVYWEIGHT_2, HEAVYWEIGHT, SNIPER)       3
86   0.003436              (FIGHTER, FIGHTER_2, FIGHTER_3)       3
96   0.002291                   (FIGHTER, THROWER, SNIPER)       3
81   0.002291             (THROWER, HEAVYWEIGHT, ASSASSIN)       3
87   0.002291            (FIGHTER, FIGHTER_2, HEAVYWEIGHT)       3
77   0.002291                 (FIGHTER, THROWER, ASSASSIN)       3
93   0.002291              (FIGHTER, THROWER, HEAVYWEIGHT)       3
90   0.002291        (HEAVYWEIGHT_2, FIGHTER, HEAVYWEIGHT)       3
108  0.001145                  (SNIPER_2, SNIPER, THROWER)       3
68   0.001145           (ASSASSIN_2, ASSASSIN_3, ASSASSIN)       3
105  0.001145            (SUPPORTER, THROWER, HEAVYWEIGHT)       3
98   0.001145  (HEAVYWEIGHT_3, HEAVYWEIGHT, HEAVYWEIGHT_2)       3
84   0.001145                  (THROWER, SNIPER, ASSASSIN)       3
112  0.001145            (SUPPORTER, SUPPORTER_2, THROWER)       3

# Check Winning Percentage for Combinations
###### We want to analyze how often a combination wins relatively to the usage of this combination. Therefore we compute the relative winning percentage for every used combination

In [25]:
length = len(df_fixed_map)
winning_percentage = []
for index, row in frequent_itemsets_combinations.iterrows():
    for idx, winner in frequent_itemsets_winner.iterrows():
        new_row = []    
        if row['itemsets'] == winner['itemsets']:
            total_number_of_combination = length * 2 * row['support']
            total_number_of_wins = length * winner['support']
            win_percentage = total_number_of_wins / total_number_of_combination
            new_row.append(winner['itemsets'])
            new_row.append(win_percentage)
            new_row.append(total_number_of_combination)
            winning_percentage.append(new_row)
winning_percentage = pd.DataFrame(winning_percentage).rename(columns={0: "itemsets", 1: 'winning percentage', 2: 'times picked'})
winning_percentage = winning_percentage[winning_percentage['itemsets'].map(len) == 3].sort_values(by=['winning percentage'], ascending=False)
winning_percentage
            

itemsets  winning percentage  times picked
87             (SNIPER_2, SNIPER, SNIPER_3)            0.748264          94.0
92        (SUPPORTER, SUPPORTER_2, THROWER)            0.689576           3.0
69              (THROWER, SNIPER, ASSASSIN)            0.689576           3.0
91    (SUPPORTER, SUPPORTER_2, SUPPORTER_3)            0.689576          21.0
84          (SNIPER_2, HEAVYWEIGHT, SNIPER)            0.646478          16.0
73             (FIGHTER, FIGHTER_2, SNIPER)            0.620619          15.0
89         (SUPPORTER, SUPPORTER_2, SNIPER)            0.525661          61.0
77          (FIGHTER, THROWER, HEAVYWEIGHT)            0.517182           4.0
67             (SNIPER_2, SNIPER, ASSASSIN)            0.509225          65.0
85         (SUPPORTER, HEAVYWEIGHT, SNIPER)            0.493127          86.0
88            (SUPPORTER, SNIPER_2, SNIPER)            0.488450         108.0
90             (SUPPORTER, THROWER, SNIPER)            0.459717           9.0
79             (SUPPORTER, FIGHTER, SNIPER)            0.448732         136.0
70       (SUPPORTER, SUPPORTER_2, ASSASSIN)            0.425915          17.0
63              (FIGHTER, SNIPER, ASSASSIN)            0.420210          64.0
68            (SUPPORTER, SNIPER, ASSASSIN)            0.399641         132.0
80               (FIGHTER, THROWER, SNIPER)            0.344788           3.0
58          (FIGHTER, ASSASSIN_2, ASSASSIN)            0.344788           6.0
72        (FIGHTER, FIGHTER_2, HEAVYWEIGHT)            0.344788           3.0
66       (SUPPORTER, HEAVYWEIGHT, ASSASSIN)            0.344788          45.0
78              (FIGHTER, SNIPER_2, SNIPER)            0.333666          31.0
81        (SUPPORTER, FIGHTER, SUPPORTER_2)            0.304225          17.0
71          (FIGHTER, FIGHTER_2, FIGHTER_3)            0.258591           4.0
74          (SUPPORTER, FIGHTER, FIGHTER_2)            0.258591           4.0
61        (SUPPORTER, ASSASSIN_2, ASSASSIN)            0.258591           8.0
83  (SUPPORTER, HEAVYWEIGHT, HEAVYWEIGHT_2)            0.258591           4.0
65          (SNIPER, HEAVYWEIGHT, ASSASSIN)            0.253314          49.0
86    (SUPPORTER, SUPPORTER_2, HEAVYWEIGHT)            0.229859           9.0
64           (SUPPORTER, FIGHTER, ASSASSIN)            0.229859          45.0
76        (SUPPORTER, FIGHTER, HEAVYWEIGHT)            0.219411          33.0
59      (ASSASSIN_2, HEAVYWEIGHT, ASSASSIN)            0.206873           5.0
82     (HEAVYWEIGHT_2, HEAVYWEIGHT, SNIPER)            0.206873           5.0
75           (FIGHTER, HEAVYWEIGHT, SNIPER)            0.206873          35.0
62         (FIGHTER, HEAVYWEIGHT, ASSASSIN)            0.167735          37.0
60           (ASSASSIN_2, SNIPER, ASSASSIN)            0.147766          21.0

# Correlation between Winning-Rate and Usage of a Combination
###### First we want to remove outliers

In [26]:
cleaned_winning_percentage = winning_percentage.copy()
def remove_outlier(df_fixed_map_in, col_name):
    q1 = df_fixed_map_in[col_name].quantile(0.25)
    q3 = df_fixed_map_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    print(fence_high)
    print(fence_low)
    df_fixed_map_out = df_fixed_map_in.loc[(df_fixed_map_in[col_name] > fence_low) & (df_fixed_map_in[col_name] < fence_high)]
    return df_fixed_map_out
remove_outlier(cleaned_winning_percentage, 'times picked')

110.0
-58.0


itemsets  winning percentage  times picked
87             (SNIPER_2, SNIPER, SNIPER_3)            0.748264          94.0
92        (SUPPORTER, SUPPORTER_2, THROWER)            0.689576           3.0
69              (THROWER, SNIPER, ASSASSIN)            0.689576           3.0
91    (SUPPORTER, SUPPORTER_2, SUPPORTER_3)            0.689576          21.0
84          (SNIPER_2, HEAVYWEIGHT, SNIPER)            0.646478          16.0
73             (FIGHTER, FIGHTER_2, SNIPER)            0.620619          15.0
89         (SUPPORTER, SUPPORTER_2, SNIPER)            0.525661          61.0
77          (FIGHTER, THROWER, HEAVYWEIGHT)            0.517182           4.0
67             (SNIPER_2, SNIPER, ASSASSIN)            0.509225          65.0
85         (SUPPORTER, HEAVYWEIGHT, SNIPER)            0.493127          86.0
88            (SUPPORTER, SNIPER_2, SNIPER)            0.488450         108.0
90             (SUPPORTER, THROWER, SNIPER)            0.459717           9.0
70       (SUPPORTER, SUPPORTER_2, ASSASSIN)            0.425915          17.0
63              (FIGHTER, SNIPER, ASSASSIN)            0.420210          64.0
80               (FIGHTER, THROWER, SNIPER)            0.344788           3.0
58          (FIGHTER, ASSASSIN_2, ASSASSIN)            0.344788           6.0
72        (FIGHTER, FIGHTER_2, HEAVYWEIGHT)            0.344788           3.0
66       (SUPPORTER, HEAVYWEIGHT, ASSASSIN)            0.344788          45.0
78              (FIGHTER, SNIPER_2, SNIPER)            0.333666          31.0
81        (SUPPORTER, FIGHTER, SUPPORTER_2)            0.304225          17.0
71          (FIGHTER, FIGHTER_2, FIGHTER_3)            0.258591           4.0
74          (SUPPORTER, FIGHTER, FIGHTER_2)            0.258591           4.0
61        (SUPPORTER, ASSASSIN_2, ASSASSIN)            0.258591           8.0
83  (SUPPORTER, HEAVYWEIGHT, HEAVYWEIGHT_2)            0.258591           4.0
65          (SNIPER, HEAVYWEIGHT, ASSASSIN)            0.253314          49.0
86    (SUPPORTER, SUPPORTER_2, HEAVYWEIGHT)            0.229859           9.0
64           (SUPPORTER, FIGHTER, ASSASSIN)            0.229859          45.0
76        (SUPPORTER, FIGHTER, HEAVYWEIGHT)            0.219411          33.0
59      (ASSASSIN_2, HEAVYWEIGHT, ASSASSIN)            0.206873           5.0
82     (HEAVYWEIGHT_2, HEAVYWEIGHT, SNIPER)            0.206873           5.0
75           (FIGHTER, HEAVYWEIGHT, SNIPER)            0.206873          35.0
62         (FIGHTER, HEAVYWEIGHT, ASSASSIN)            0.167735          37.0
60           (ASSASSIN_2, SNIPER, ASSASSIN)            0.147766          21.0

###### As we see no entries are removed

In [27]:
print(len(winning_percentage))
print(len(cleaned_winning_percentage))

35
35


###### Next we want to check whether there is a correlation between usage of a combination and the winning rate.

In [33]:
cleaned_winning_percentage['times picked'].corr(cleaned_winning_percentage['winning percentage'])

0.20754273476114093

As wee see the correlation is slightly positive with 0.208. Thus, there is a small correlation between the usage of a combination of brawler types and the winning rate. This indicates that it is more probable to win with a certain combination of brawler types on specific maps. it is also probable that some combinations are used very frequently and there fore team 1 and team 2 use the same combination of brawler types which then lowers our computed correlation.